In [2]:
import os
import numpy as np
import shutil
import glob
import math

In [8]:
glob.glob("*0")[0][0]

'2'

In [6]:
gbnames = [int(i) for i in glob.glob("*0")]
print(gbnames)

[210, 310, 410, 510, 610, 710, 810, 910]


In [9]:
gbtans = [float(i[1])/float(i[0]) for i in glob.glob("*0")]

In [14]:
alpha = np.arctan(1/2)

In [15]:
keikakus = [np.arctan(i) for i in gbtans]

In [18]:
thetas = alpha -keikakus

In [20]:
gbcos = [np.cos(i) for i in thetas]

In [21]:
gbcos

[1.0,
 0.9899494936611666,
 0.9761870601839527,
 0.9647638212377322,
 0.9557790087219501,
 0.9486832980505138,
 0.9429903335828895,
 0.9383431168171101]

In [29]:
lattice_constant = 3.61999999500576
alpha = np.arctan(1/2)
for gbname in glob.glob("*0"):
    
    dy = lattice_constant/2
    keikaku = np.arctan(float(gbname[1])/float(gbname[0]))
    theta = alpha - keikaku
    gbcos = np.cos(theta)
    dz = -np.sqrt(5)*lattice_constant*gbcos/2
    with open(f"./{gbname}/proto.in","w") as f:
        f.write(f"""

variable tx equal 0 # displacement in x direction
variable ty equal {dy} # displacement in y direction
variable tz equal {dz} # displacement in z direction

# end end of defining looping parameter------------------



clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile2
include ./blockfile

mass 1 63.546 #Cu
mass 2 63.546 #Cu

# ---------- Define Interatomic Potential --------------------- 
pair_style meam
pair_coeff * * library.Cu.meam Cu Cu.meam Cu Cu
neighbor 2.0 bin 
neigh_modify delay 10 check yes 

# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle reduce sum c_eng
compute bulk_dis bulk displace/atom 
compute hulk_dis_ave_x bulk reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in Cu crystal 
variable minimumenergy equal -3.54000000004256
variable gbarea equal "ly * lz" 

# ---------- Run Minimization ---------------------
reset_timestep 0

displace_atoms right move ${{tx}} ${{ty}} ${{tz}} units box

velocity fixbulk1 zero linear
fix fixbulk1 fixbulk1 setforce 0.0 0.0 0.0

velocity fixbulk2 zero linear
fix fixbulk2 fixbulk2 setforce NULL 0.0 0.0

min_style cg 
minimize 1e-25 1e-25 5000 10000

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x
dump            1 all custom 1 final id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x


#----------- output calculation result of each loop into results file 
print "Grain Boundary energy (meV) = ${{gbemJm2}};"
print "All done!" 






            """)